In [ ]:
import numpy as np
from hwcomponents import DataStructures as ds 
from hwcomponents import Utils

In [ ]:
# Test Neuron datastructure
ds.Neuron(
    param_leak_str=1,
    param_threshold=20,
    param_reset=0,
    state_core=0
)

In [ ]:
# Test Synapse datastructure
ds.Synapse(
    length=10,
    word_size=np.uint8
)

In [ ]:
# ex of random neuron generator
def random_neuron() -> ds.Neuron:
    return ds.Neuron(
        param_leak_str=Utils.DiscreteNORM(2**3, 2, 0, 2**5),
        param_threshold=Utils.DiscreteNORM(2**6, 2**3, 0, 2**11),
        state_core=0,
        param_reset=Utils.DiscreteNORM(4., 1., 0, 2**3)
    )

random_neuron()

In [ ]:
# ex of BRAM with random neurons
from hwcomponents import BRAM as bram

amount_of_neurons = 10
neuron_generator = lambda: random_neuron()

example_bram_for_neurons = bram.Neurons(
    length=amount_of_neurons,
    randomize_fun=neuron_generator
)

for i in range(example_bram_for_neurons.length):
    print(example_bram_for_neurons[i])

In [ ]:
# ex of random synapse generator
# generate a synapse register for one neuron. 
# in this case it generates a 2-bit synaptic weight (0..3 in decimal)

def random_synapse(depth: int) -> ds.Synapse:
    temp_synapse = ds.Synapse(depth, np.int8)
    for i in range(depth):
        temp_synapse[i] = Utils.DiscreteNORM(0.5, 1., 0, 3)
    return temp_synapse

In [ ]:
# ex of BRAM with random synapses, lets make 10 synapses for 10 neurons
from hwcomponents import BRAM as bram
amount_of_synapses = amount_of_neurons = 10

example_of_bram_for_synapses = bram.Synapses(
    length=amount_of_neurons,
    depth=amount_of_synapses,
    word_size=np.int8,
    randomize_fun=random_synapse #the generator function must have a depth parameter
)
for i in range(example_of_bram_for_synapses.length):
    print(example_of_bram_for_synapses[i])

In [ ]:
# LIF example
from hwcomponents import Neurons as N
active_neuron = N.LIF() # Lif neuron object 

# Update Neuron state, expects a DataStructures.Neuron object
# Lets grab one of the previously generated neurons
neuron_idx, weight_idx = 0, 5

neurons = example_bram_for_neurons
synapses = example_of_bram_for_synapses

active_neuron.change_neuron(neuron=neurons[0])
active_neuron.change_weight(synapses[neuron_idx][weight_idx])
Utils.Print.PrettyNeuron(neurons[0])
print("\nSynapse weight: ", synapses[neuron_idx][weight_idx], "\n")

# Simulate Spike event on input synapse
active_neuron.forward(
    input = 1
)

# Print the updated neuron
Utils.Print.PrettyNeuron(neurons[0])

In [ ]:
# Lets manage some inputs. They are in the DataStructures namespace
from hwcomponents import DataStructures as ds

# Lets make a random generator
def random_input_sample(length: int) -> np.array:
    return np.array([
        Utils.DiscreteNORM(.1, .5, -1, 1) for _ in range(length)
    ])

# Lets make a random input sample

ex_sample = ds.InputSample(
    length=10,
    randomize_fun=random_input_sample
)

print(ex_sample)

In [ ]:
# We can subclass the InputSample class to make a custom input sample
# We can put our inputsamples into collections
from typing import Callable

def generate_random_samples(
        amount_of_samples: int,
        sample_length: int,
        sample_random_gen: Callable[[int], np.ndarray]
):
    data_collection = ds.InputData() # instantiate the data wrapper
    for _ in range(amount_of_samples):
        data_collection.add_sample(
            ds.InputSample(sample_length, randomize_fun=sample_random_gen)
        )
    return data_collection

test_data = generate_random_samples(10, 10, random_input_sample)

# lets make a heatmap of the data
for i in range(len(test_data)):
    print(test_data[i])


In [ ]:
Utils.Plotting.input_data_as_heatmap(test_data)